# Laboratorio de Amazon Rekognition

En este laboratorio exploraremos las siguientes capacidades de Rekognition.

- Detección de Objetos y Escenas en imágenes.
- Análisis Facial
- Comparación de rostros
- Detección de equipo protector

En este laboratorio usaremos las siguientes librerías de Python.

- SDK de AWS para Python - [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)
- Python Requests - [requests](https://docs.python-requests.org/en/master/)
- Python IO - [io](https://docs.python.org/3/library/io.html)
- Jupyter IPython - [IPython](https://ipython.readthedocs.io/en/stable/)
- Python Pillow - [PIL](https://pillow.readthedocs.io/en/stable/)

*Tip: Durante los laboratorios es posible cambiar las URLs de las imagenes para experimentar con otros archivos.*

## Detección de Objetos y Escenas

En esta sección tomaremos diferentes imágenes y las analizaremos utilizando el servicio de Rekognition.

Empezaremos por importar las librerías necesarias para la manipulación de imágenes y el cliente para utilizar Rekognition.

In [ ]:
import boto3
import requests
import json
import io
from IPython.display import Image, display, JSON, Markdown, HTML
from PIL import Image as PImage, ImageDraw, ImageFont

# Se obtiene el cliente de Rekognition.
rekognition = boto3.client('rekognition')

Esta función utilitaria nos servirá para crear rectángulos al rededor de los objetos detectados.

In [ ]:
def box_labels(image_bytes, labels, confidence):
    font = ImageFont.load_default()
    image = PImage.open(io.BytesIO(image_bytes))
    width, height = image.size
    draw = ImageDraw.Draw(image)
    for label in labels:
        name = label['Name']
        if label['Confidence'] >= confidence:
            for instance in label['Instances']:
                if instance['Confidence'] >= confidence and 'BoundingBox' in instance:
                    start_x = instance['BoundingBox']['Left'] * width
                    start_y = instance['BoundingBox']['Top'] * height
                    end_x = start_x + instance['BoundingBox']['Width'] * width
                    end_y = start_y + instance['BoundingBox']['Height'] * height
                    draw.rectangle([(start_x, start_y), (end_x, end_y)], outline='black', width=2)
                    draw.text((start_x + 5, start_y + 5), name, fill='black', font=font)
    image_out = io.BytesIO()
    image.save(image_out, 'JPEG')
    return image_out.getvalue()

A continuación descargaremos las imágenes que serán utilizadas para detectar objetos y para detectar la escena.

In [ ]:
objects_url = 'https://dhei5unw3vrsx.cloudfront.net/images/skateboard_resized.jpg'
scene_url = 'https://dhei5unw3vrsx.cloudfront.net/images/landscape_resized.jpg'
objects_response = requests.get(objects_url, allow_redirects=True)
scene_response = requests.get(scene_url, allow_redirects=True)

Ahora que contamos con las imágenes a analizar comencemos con la detección de objetos.

In [ ]:
objects_image = objects_response.content
display(Image(data=objects_image))

Rekognition cuenta con la funcionalidad de detección de etiquetas, para mayor detalles
podemos visitar la sección de
 [detect_labels](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.detect_labels)
 en la documentación. En este caso enviaremos directamente los bytes de la imagen.

In [ ]:
objects_a = rekognition.detect_labels(Image={
    'Bytes': objects_image
})
display(Markdown(data='### Respuesta de Rekognition'))
json_text = '''```
{}
```'''.format(json.dumps(objects_a, indent=2))
display(Markdown(data=json_text))

In [ ]:
## Set a high confidence to reduce number of boxes.
new_objects_image = box_labels(objects_image, objects_a['Labels'], 90.0)
display(Markdown(data='### Imagen con objetos detectados'))
display(Image(data=new_objects_image))

Ahora analizaremos el contexto de la siguiente imagen.

In [ ]:
scene_image = scene_response.content
display(Image(data=scene_image))

Utilizando el mismo servicio obtenemos el contexto.

In [ ]:
objects_a = rekognition.detect_labels(Image={
    'Bytes': scene_image
})
json_text = '''```
{}
```'''.format(json.dumps(objects_a, indent=2))
display(Markdown(data=json_text))

In [ ]:
confidence = 90.0
html_text = '<table><tr><th>Etiqueta</th><th>Confianza</th></tr>'
for label in objects_a['Labels']:
    if label['Confidence'] >= confidence:
        html_text += '<tr><td>{}</td><td>{}</td></tr>'.format(label['Name'], label['Confidence'])
html_text += '</table>'
display(HTML(data=html_text))

## Análisis facial

Ahora realizaremos un análisis facial.

In [ ]:
single_face_url = 'https://dhei5unw3vrsx.cloudfront.net/images/drive_resized.jpg'
multi_face_url = 'https://dhei5unw3vrsx.cloudfront.net/images/family_resized.jpg'
single_face_image = requests.get(single_face_url, allow_redirects=True).content
multi_face_image = requests.get(multi_face_url, allow_redirects=True).content

Definiremos las siguientes funciones para crear un cuadro donde Rekognition detecto el rostro e imprimir los detalles de cada rostro.

In [ ]:
box_colors = ['red', 'green', 'blue', 'orange', 'yellow', 'black', 'grey', 'purple']

def box_face(image_bytes, faces):
    image = PImage.open(io.BytesIO(image_bytes))
    width, height = image.size
    draw = ImageDraw.Draw(image)
    idx = 0
    for face in faces:
        if 'BoundingBox' in face:
            start_x = face['BoundingBox']['Left'] * width
            start_y = face['BoundingBox']['Top'] * height
            end_x = start_x + face['BoundingBox']['Width'] * width
            end_y = start_y + face['BoundingBox']['Height'] * height
            draw.rectangle([(start_x, start_y), (end_x, end_y)], outline=box_colors[idx], width=2)
            idx = idx + 1 if idx + 1 < len(box_colors) else 0
    image_out = io.BytesIO()
    image.save(image_out, 'JPEG')
    return image_out.getvalue()

def features(faces, confidence):
    idx = 0
    markdown_text = ''
    for face in faces:
        markdown_text += '\n#### Caja color ' + box_colors[idx]
        markdown_text += '\n Rango de edad entre {} y {}'.format(face['AgeRange']['Low'], face['AgeRange']['High'])
        markdown_text += '\n\n| Característica | Valor | Confianza |'
        markdown_text += '\n| -------------- | ----- | --------- |'
        markdown_text += '\n| {} | {} | {} |'.format('Sonriendo', face['Smile']['Value'], face['Smile']['Confidence'])
        markdown_text += '\n| {} | {} | {} |'.format('Lentes', face['Eyeglasses']['Value'], face['Eyeglasses']['Confidence'])
        markdown_text += '\n| {} | {} | {} |'.format('Lentes de sol', face['Sunglasses']['Value'], face['Sunglasses']['Confidence'])
        markdown_text += '\n| {} | {} | {} |'.format('Sexo', face['Gender']['Value'], face['Gender']['Confidence'])
        markdown_text += '\n| {} | {} | {} |'.format('Barba', face['Beard']['Value'], face['Beard']['Confidence'])
        markdown_text += '\n| {} | {} | {} |'.format('Bigote', face['Mustache']['Value'], face['Mustache']['Confidence'])
        markdown_text += '\n| {} | {} | {} |'.format('Ojos abiertos', face['EyesOpen']['Value'], face['EyesOpen']['Confidence'])
        markdown_text += '\n| {} | {} | {} |'.format('Boca abierta', face['MouthOpen']['Value'], face['MouthOpen']['Confidence'])
        for emotion in face['Emotions']:
            if emotion['Confidence'] >= confidence:
                markdown_text += '\n\nLa persona esta {}'.format(emotion['Type'])
        idx = idx + 1 if idx + 1 < len(box_colors) else 0
    return markdown_text

En este primer ejercicio utilizaremos una imagen con un solo rostros para el análisis.
Para mayor información de los parámetros podemos visitar la sección de
 [detect_faces](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.detect_faces)
 en la documentación.

In [ ]:
display(Image(data=single_face_image))

In [ ]:
detect_face_r = rekognition.detect_faces(Image={'Bytes': single_face_image}, Attributes=['ALL'])
display(Image(data=box_face(single_face_image, detect_face_r['FaceDetails'])))
display(Markdown(data=features(detect_face_r['FaceDetails'], 80.0)))

Ahora realicemos el análisis sobre una imagen con diferentes rostros.

In [ ]:
display(Image(data=multi_face_image))

In [ ]:
detect_face_r = rekognition.detect_faces(Image={'Bytes': multi_face_image}, Attributes=['ALL'])
display(Image(data=box_face(multi_face_image, detect_face_r['FaceDetails'])))
display(Markdown(data=features(detect_face_r['FaceDetails'], 80.0)))

## Comparación de Rostros

En este laboratorio haremos una comparación de rostros entre dos imágenes con la finalidad de detectar si una persona se encuentra en ambas fotografías.

In [ ]:
source_img_url = 'https://dhei5unw3vrsx.cloudfront.net/images/source3_resized.jpg'
target_img_url = 'https://dhei5unw3vrsx.cloudfront.net/images/target3_resized.jpg'
source_image = requests.get(source_img_url, allow_redirects=True).content
target_image = requests.get(target_img_url, allow_redirects=True).content

display(Image(data=source_image))
display(Image(data=target_image))

Definimos un par de funciones utilitarias para poder realizar un crop de las secciones de la imagen que contienen rostros.

In [ ]:
def crop_face(image_bytes, face):
    image = PImage.open(io.BytesIO(image_bytes))
    width, height = image.size
    cropped_bytes = image_bytes
    if 'BoundingBox' in face:
        start_x = face['BoundingBox']['Left'] * width
        start_y = face['BoundingBox']['Top'] * height
        end_x = start_x + face['BoundingBox']['Width'] * width
        end_y = start_y + face['BoundingBox']['Height'] * height
        cropped = image.crop((start_x, start_y, end_x, end_y))
        image_out = io.BytesIO()
        cropped.save(image_out, 'JPEG')
        cropped_bytes = image_out.getvalue()
    return cropped_bytes

def crop_faces(image_bytes, faces):
    cropped_faces = list()
    image = PImage.open(io.BytesIO(image_bytes))
    width, height = image.size
    for face in faces:
        if 'Face' in face and 'BoundingBox' in face['Face']:
            start_x = face['Face']['BoundingBox']['Left'] * width
            start_y = face['Face']['BoundingBox']['Top'] * height
            end_x = start_x + face['Face']['BoundingBox']['Width'] * width
            end_y = start_y + face['Face']['BoundingBox']['Height'] * height
            cropped = image.crop((start_x, start_y, end_x, end_y))
            image_out = io.BytesIO()
            cropped.save(image_out, 'JPEG')
            cropped_faces.append(image_out.getvalue())
        elif 'BoundingBox' in face:
            start_x = face['BoundingBox']['Left'] * width
            start_y = face['BoundingBox']['Top'] * height
            end_x = start_x + face['BoundingBox']['Width'] * width
            end_y = start_y + face['BoundingBox']['Height'] * height
            cropped = image.crop((start_x, start_y, end_x, end_y))
            image_out = io.BytesIO()
            cropped.save(image_out, 'JPEG')
            cropped_faces.append(image_out.getvalue())
    return cropped_faces

Ahora realizamos el análisis con Rekognition, utilizaremos el servicio
 [compare_faces](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.compare_faces).

In [ ]:
response = rekognition.compare_faces(SourceImage={'Bytes': source_image},
                                     TargetImage={'Bytes': target_image}) # Set to zero to recover al faces.
display(Markdown(data='### Rostro en la imagen origen'))
display(Image(data=crop_face(source_image, response['SourceImageFace'])))
display(Markdown(data='### Rostro localizado en la imagen destino'))
for face in crop_faces(target_image, response['FaceMatches']):
    display(Image(data=face))
display(Markdown(data='### Rostros sin parecido en imagen destino'))
for face in crop_faces(target_image, response['UnmatchedFaces']):
    display(Image(data=face))

## Detección de equipo protector

En este laboratorio utilizaremos Rekognition para detectar equipo protector en imágenes.

In [ ]:
ppe_img_url = 'https://dhei5unw3vrsx.cloudfront.net/images/ppe_group_updated.jpg'
ppe_image = requests.get(ppe_img_url, allow_redirects=True).content
display(Image(data=ppe_image))

Definiremos una función utilitaria para enmarcar el equipo protector detectado en la imagen.

In [ ]:
def ppe_box(image_bytes, persons):
    image = PImage.open(io.BytesIO(image_bytes))
    width, height = image.size
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    for person in persons:
        for body_part in person['BodyParts']:
            name = body_part['Name']
            for equipment in body_part['EquipmentDetections']:
                start_x = equipment['BoundingBox']['Left'] * width
                start_y = equipment['BoundingBox']['Top'] * height
                end_x = start_x + equipment['BoundingBox']['Width'] * width
                end_y = start_y + equipment['BoundingBox']['Height'] * height
                draw.rectangle([(start_x, start_y), (end_x, end_y)], outline='yellow', width=2)
                draw.text((start_x + 5, start_y + 5), name + ' ' + equipment['Type'], fill='black', font=font)
    image_out = io.BytesIO()
    image.save(image_out, 'JPEG')
    return image_out.getvalue()


Ahora realizaremos el análisis de la imagen y marcaremos el equipo detectado con el servicio
 [detect_protective_equipment](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.detect_protective_equipment).

In [ ]:
ppe_response = rekognition.detect_protective_equipment(Image={'Bytes': ppe_image})
display(Image(data=ppe_box(ppe_image, ppe_response['Persons'])))